In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from src.utilities import set_matplotlib_defaults
import plotly.io as pio
# Set default plotly theme
pio.templates.default = 'plotly_white'
set_matplotlib_defaults()

from src.utilities import Matrix

from src.observables import *


SUFFIX = '.npy'
OMEGA = 1  # Interaction Strength

HEAT = False

### Functions

In [ ]:
def single_entropy(cov, system):
    d1, d2 = symplectic_eigenvalues(cov)
    if system == 1:
        x1 = d1 + 1/2
        x2 = d1 - 1/2
    elif system == 2:
        x1 = d2 + 1/2
        x2 = d2 - 1/2
    return x1 * np.log(x1) - x2 * np.log(x2)

In [ ]:
def get_observable(observable, covs, iterator):
    if observable == 'Entropy':
        func = symplectic_entropy
    elif observable == 'Mutual Information':
        func = mutual_information
    elif observable == 'Quantum Discord':
        func = gaussian_quantum_discord
    elif observable == 'Logarithmic Negativity':
        func = logarithmic_negativity 
    else:
        raise ValueError(f'Observable {observable} not supported')

    arr = []
    for i in iterator:
        arr.append(func(covs[i]))
    return np.array(arr).real

In [ ]:
observables = ['Temperatures', 'Entropies', 'Mutual Information', 'Quantum Discord', 'Logarithmic Negativity', 'Heat Flux']
steps_per_timedelta = 1

steps = 10000  # To filter data points

def extract_data(observables, indx, metadata, verbose=False):
    dt_df = pd.DataFrame()
    timedelta = metadata['timedelta']
    omega = metadata['omega']
    
    cov_filename = f'../objects/saved/{indx}_rho_covariance.npy'
    cov_evolution = np.load(cov_filename)

    if verbose:
        print(f'{len(cov_evolution):>21}\t'
              f'{indx:>10}\t'
              f'{timedelta:>21}\t'
              f'{omega:>21}')

    # Iterators to select only a sample of data
    iterator = range(len(cov_evolution) - 1)  # np.linspace(0, len(cov_evolution) - 1, steps).astype(int)
    
    dt_df['Log ID'] = [indx for _ in iterator]
    dt_df['Interaction Step'] = [i for i in iterator]
    dt_df['Interaction Time'] = [timedelta for i in iterator] 
    

    for obs in observables:
        if obs == 'Symplectic':
            dt_df['d1'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
            dt_df['d2'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
            dt_df['d1-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real
            dt_df['d2-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real
        elif obs == 'Entropies':
            dt_df['S'] = np.array([symplectic_entropy(cov_evolution[i]) for i in iterator]).real
            dt_df['S1'] = np.array([single_entropy(cov_evolution[i], 2) for i in iterator]).real
            dt_df['S2'] = np.array([single_entropy(cov_evolution[i], 1) for i in iterator]).real
        elif obs == 'Photons':
            dt_df['N1'] = np.array([mean_photon_numbers(cov_evolution[i])[0] for i in iterator]).real
            dt_df['N2'] = np.array([mean_photon_numbers(cov_evolution[i])[1] for i in iterator]).real
        elif obs == 'Temperatures':
            dt_df['T1'] = np.array([mean_temperatures(cov_evolution[i])[0] for i in iterator]).real
            dt_df['T2'] = np.array([mean_temperatures(cov_evolution[i])[1] for i in iterator]).real
            dt_df['T_ratio'] = dt_df['T1'].values / dt_df['T2'].values
        elif obs == 'Quantum Discord':
            dt_df['DL'] = np.array([gaussian_quantum_discord(cov_evolution[i])[0] for i in iterator]).real
            dt_df['DR'] = np.array([gaussian_quantum_discord(cov_evolution[i])[1] for i in iterator]).real
        elif obs == 'Phase':
            try:
                random_phis = np.load(f'../objects/saved/{indx}_rho_phis.npy')
                dt_df['phi'] = random_phis
            except FileNotFoundError:
                print(f'Could not find file {indx}_rho_phis.npy')
        elif obs == 'Times':
            try:
                random_times = np.load(f'../objects/saved/{indx}_rho_gaussian_times.npy')
                dt_df['Interaction Time'] = random_times
            except FileNotFoundError:
                print(f'Could not find file {indx}_rho_gaussian_times.npy')
        else:
            dt_df[obs] = get_observable(obs, cov_evolution, iterator)
            
    return dt_df

# Effects of Stochastic Interaction Times (1) and Phaseonium Phases (2)


For (1) use data with IDs with R like 0R00Th and for (2) use data with IDs with P like 0P00Th. 

In [ ]:
log = pd.read_csv('../objects/saved/logs.csv', sep=';', decimal='.', index_col='Id')
# Format the index values as strings with leading zeros
log.index = log.index.astype(str).str.zfill(3)
log.tail()

In [ ]:
plt.close('all')

def make_plots(total_df, observables, layout=(2, 2), figsize=(10, 4), **kwargs):
    # Make a 2x4 plot, one row per Final Temperature and one column per Interaction Time 
    fig_temps, axs_temps = plt.subplots(nrows=layout[0], ncols=layout[1], figsize=figsize, layout='tight')
    axs_temps = axs_temps.flatten()
    fig_temps.suptitle('Temperatures of the cavities $S1$ and $S2$')
    # Temperatures Ratio
    fig_ratio, axs_ratio = plt.subplots(nrows=layout[0], ncols=layout[1], figsize=figsize, layout='tight')
    axs_ratio = axs_ratio.flatten()
    fig_ratio.suptitle('Temperatures Ratio $T_{S1}/T_{S2}$')
    # Random Times / Phases
    fig_rand, axs_rand = plt.subplots(nrows=layout[0], ncols=layout[1], figsize=figsize, layout='tight')
    axs_rand = axs_rand.flatten()
    
    ids = total_df['Log ID'].unique()
    
    for i, stem in enumerate(ids):
        log_id = stem if stem[0] != 'R' else "1" + stem
        
        ax_temps = axs_temps[i]
        ax_ratio = axs_ratio[i]
        ax_rand = axs_rand[i]
        
        df = total_df.loc[total_df['Log ID'] == stem]
        dt = log.loc[log_id, 'timedelta']
        tf = log.loc[log_id, 'Tf']
        
        ax_temps.plot(df['Interaction Step'], df[('T1', 'mean')], label=f'{dt}, {tf}')
        ax_temps.plot(df['Interaction Step'], df[('T2', 'mean')], label=f'{dt}, {tf}')
        # Plot the standard deviation as a colored area
        ax_temps.fill_between(df['Interaction Step'], 
                        df[('T1', 'mean')] - df[('T1', 'std')], 
                        df[('T1', 'mean')] + df[('T1', 'std')], 
                        color='b', alpha=0.2)
        ax_temps.fill_between(df['Interaction Step'], 
                        df[('T2', 'mean')] - df[('T2', 'std')], 
                        df[('T2', 'mean')] + df[('T2', 'std')], 
                        color='r', alpha=0.2)
        # Plot the ratio
        ax_ratio.plot(df['Interaction Step'], df[('T_ratio', 'mean')], label=f'{dt}, {tf}')
        ax_ratio.fill_between(df['Interaction Step'], 
                        df[('T_ratio', 'mean')] - df[('T_ratio', 'std')], 
                        df[('T_ratio', 'mean')] + df[('T_ratio', 'std')], 
                        color='b', alpha=0.2)
        
        if 'Phase' in observables:
            ax_rand.plot(df[('phi', 'mean')], label=f'{stem}')
        elif 'Times' in observables:
            ax_rand.plot(df[('Interaction Time', 'mean')], label=f'{stem}')
        
        for ax in [ax_temps, ax_ratio, ax_rand]:
            if kwargs.get('legend', False):
                ax.legend()
            
            # Set column names
            if i==0 or i==1:
                ax.set_title(f'$\Delta t = {dt}, {stem}$')
            # Set row names
            if i==0 or i==2:
                ax.set_ylabel(f"$T_{'{st}'} = {tf}, {stem}$")
                    
    
    return fig_temps, fig_ratio


In [ ]:
def aggregate_stochastic_data(df_dict, observables, max_len=0):
    total_df = pd.DataFrame()

    for i, (stem, ids) in enumerate(df_dict.items()):
        try:
            df = pd.concat([extract_data(observables, indx, log.loc[indx]) for indx in ids])
        except FileNotFoundError as e:
            print(f'Could not find data for ID: {stem}: {e}')
        except KeyError as e:
            print(f'Could not load data for ID: {stem}: {e}')
            continue
        else:
            df = df.drop(columns=['Log ID']).groupby('Interaction Step').agg(['mean', 'std'])
            print(stem)
            df['Log ID'] = stem
            
            if max_len > 0:
                df = df.reindex(np.arange(1, max_len + 1), method='ffill')
                
            for temperature in ['T1', 'T2']:
                df[f'{temperature}min'] = df[(temperature, 'mean')] - df[(temperature, 'std')]
                df[f'{temperature}max'] = df[(temperature, 'mean')] + df[(temperature, 'std')]
            df.reset_index(inplace=True)
            total_df = pd.concat([total_df, df])
    
    return total_df

In [ ]:
def set_tore_layout(axs, **kwargs):
    linestyle = dict(linestyle=(0, (7, 5)), linewidth=1, color='k', alpha=0.5)
    plots_linestyle = dict(linestyle='-', linewidth=4, zorder=5)
    
    xlim = kwargs.get('xlim', (-7, 260))
    
    for ax in axs:
        ax.set_ylim(0.45, 1.55)
        ax.set_xlim(xlim)
        
        ax.plot(list(xlim), [1, 1], **linestyle)
        ax.plot(list(xlim), [1.5, 1.5], **linestyle)
        ax.plot(list(xlim), [0.5, 0.5], **linestyle)
        
        ax.grid(visible=False)
        ax.set_yticks([0.6, 0.8, 1.0, 1.2, 1.4])
        
        ax.set_xlabel('number of collisions', fontsize=32)

        ax.tick_params(axis='both', which='major', colors='black', direction='in', labelsize=28)
        # Change the color of the frame borders
        for spine in ax.spines.values():
            spine.set_color('black')
    
    axs[0].set_ylabel('temperature', fontsize=32)
    return axs, plots_linestyle

def make_tore_plots(df, colormap, figsize=(10, 5), **kwargs):
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=figsize, layout='tight')
    axs = axs.flatten()
    axs, linestyle = set_tore_layout(axs, **kwargs)
    
    ids = df['Log ID'].unique()
    
    for i, stem in enumerate(ids):
        df_to_plot = df.loc[df['Log ID'] == stem]
        color = colormap[stem]
        
        steps = df_to_plot['Interaction Step']  # x-axis
        for ax, temperature in zip(axs, ['T1', 'T2']):
            if 'R' in stem:
                # Random simulations must be plotted with their standard deviation
                ax.plot(steps, df_to_plot[(temperature, 'mean')], label=f'{stem}', 
                        color=color, **linestyle)
                ax.fill_between(steps, df_to_plot[f'{temperature}min'], df_to_plot[f'{temperature}max'], 
                                color=color, alpha=0.5, zorder=10)
            else:
                ax.plot(steps, df_to_plot[(temperature, 'mean')], label=f'{stem}', color=color, lw=4, ls=(0, (1, 2)), zorder=2)
    return fig

plt.close('all')

## Recap Plot

In [ ]:
dt_tf_dict = {
    '01Th': ["01Th"],
    'R01Th': [f"{i}R01Th" for i in range(10)],
    '05Th': ["05Th"],
    'R05Th': [f"{i}R05Th" for i in range(10)],
}

observables = ['Temperatures', 'Quantum Discord', 'Times']

df_times = aggregate_stochastic_data(dt_tf_dict, observables)
df_times.head()

In [ ]:
fig_temps, fig_ratio = make_plots(df_times, observables, layout=(2, 2), figsize=(10, 6))

In [ ]:
fig_temps.savefig('../img/obs_evolution/stochastic/randomphases_temperatures.png', dpi=300)
fig_ratio.savefig('../img/obs_evolution/stochastic/randomphases_tempratio.png', dpi=300)

## Paper Plots

In [ ]:
colormap = {
    '01Th': 'cornflowerblue',
    'R01Th': 'blue',
    '05Th': 'lightsalmon',
    'R05Th': 'crimson',
}

fig_tore = make_tore_plots(df_times, colormap, figsize=(16, 6))

In [ ]:
fig_tore.savefig('../img/obs_evolution/stochastic/randomtimes_temperature.png', dpi=300)
fig_tore.savefig('../img/obs_evolution/stochastic/randomtimes_temperature.pdf', dpi=300)

In [ ]:
dt_tf_dict = {
    '37Th': ["37Th"],
    'R37Th': [f"{i}R37Th" for i in range(10)],
    '33Th': ["33Th"],
    'R33Th': [f"{i}R33Th" for i in range(10)],
}

observables = ['Temperatures', 'Quantum Discord', 'Phase']

df_phases = aggregate_stochastic_data(dt_tf_dict, observables, max_len=1500)
df_phases.head()

In [ ]:
df_phases["Log ID"].unique()

In [ ]:
colormap = {
    '33Th': 'dodgerblue',
    'R33Th': 'blue',
    '37Th': 'lightsalmon',
    'R37Th': 'crimson',
}

fig_tore_phases = make_tore_plots(df_phases, colormap, figsize=(16, 6), xlim=(-20, 1510))

In [ ]:
plt.xlim(0,500)
plt.show()

In [ ]:
fig_tore_phases.savefig('../img/obs_evolution/stochastic/randomphases_temperature.png', dpi=300)
fig_tore_phases.savefig('../img/obs_evolution/stochastic/randomphases_temperature.pdf', dpi=300)

In [ ]:
df_phases['Tspread1'] = df_phases['T1max'] - df_phases['T1min']
df_phases['Tspread2'] = df_phases['T2max'] - df_phases['T2min']

fig_spread, ax = plt.subplots(figsize=(10, 4))
df_phases.loc[df_phases['Log ID'] == 'R37Th', 'Tspread1'].plot(ax=ax)
df_phases.loc[df_phases['Log ID'] == 'R37Th', 'Tspread2'].plot(ax=ax)

In [ ]:
recap = pd.DataFrame(columns=pd.MultiIndex.from_product([['R01Th', 'R05Th', 'R33Th', 'R37Th'], ['T1', 'T2', 'Tspread1', 'Tspread2']]))
megadata = pd.concat([df_times, df_phases])
for stem, thermal_time in zip(['R01Th', 'R05Th', 'R33Th', 'R37Th'], [(100, 300), (200, 300), (300, 350), (1200, 1500)]):
    data = megadata.loc[
        (megadata['Log ID'] == stem) & 
        (megadata['Interaction Step'] > thermal_time[0]) &
        (megadata['Interaction Step'] < thermal_time[1])
    ]
    recap[(stem, 'Tspread1')] = data['Tspread1'].describe()
    recap[(stem, 'Tspread2')] = data['Tspread2'].describe()
    recap[(stem, 'T1')] = data[('T1', 'mean')].describe()
    recap[(stem, 'T2')] = data[('T2', 'mean')].describe()

stem = 'R33Th'
mean_temp = recap[[(stem, 'T1'), (stem, 'T2')]].mean(axis=1)['mean']
mean_dev = recap[[(stem, 'Tspread1'), (stem, 'Tspread2')]].mean(axis=1)['mean']
print(round(mean_temp, 3), ' +- ', round(mean_dev, 3))

In [ ]:
for dataframe, stem in zip([df_times, df_phases],[['R01Th', 'R05Th'], ['R33Th', 'R37Th']]):
    print(stem)
    print(dataframe)

In [ ]:
fig_n, ax = plt.subplots()
for indx, group in df.groupby('Log ID'):
    print(indx)
    timedelta = log.loc[indx, 'timedelta']
    label = f"{log.loc[indx, 'alpha']} + {log.loc[indx, 'phi']}"
    ax.plot(group['Time'], group['S'], label=label)
ax.legend()
ax.set_xlim(0, 250)
plt.draw()
plt.show()

In [ ]:

fig_n, ax = plt.subplots()

for indx, group in df.groupby('Log ID'):
    timedelta = log.loc[indx, 'timedelta']
    label = f"{log.loc[indx, 'alpha']} + {log.loc[indx, 'phi']}"
    ax.plot(group['Time'], group['Quantum Discord'], label=label)
ax.legend()
ax.set_xlim(0, 200)
plt.draw()
plt.show()

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Log ID'], value_vars=['N1', 'N2'], var_name='System', value_name='Temperature')
fig_temp = px.line(df_melt, x='Time', y='Temperature',
                   color='Log ID', line_dash='System',
                   title='Mean Photon Numbers in the Systems',
                   labels={'Temperature': 'Mean Photon '
                                          'Number', 'Time': 'Time (a.u.)'})
fig_temp.update_xaxes(range=[0, 2000])
fig_temp.show()

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Log ID'], value_vars=['S1', 'S2'], var_name='System', value_name='Entropy')
fig_entropy = px.line(df_melt, x='Time', y='Entropy',
                   color='Log ID', line_dash='System',
                   title='Systems Entropies')
fig_entropy.update_xaxes(range=[0, 200])
fig_entropy.show()

In [ ]:
fig_discord = px.line(df, x='Time', y='Quantum Discord', color='Log ID', title='Quantum Discord')
fig_discord.update_xaxes(range=[0, 200])
fig_discord.show()

In [ ]:
fig_mutinf = px.line(df, x='Time', y='Mutual Information', color='Log ID', title='Mutual Information')
fig_mutinf.update_xaxes(range=[0, 1500])
fig_mutinf.show()

In [ ]:
fig_logneg = px.line(df, x='Time', y='Logarithmic Negativity', color='Log ID', title='Logarithmic Negativity')
fig_logneg.update_xaxes(range=[0, 1500])
fig_logneg.show()

In [ ]:
df_melt = df.loc[df['Time']>0].melt(id_vars=['Time', 'Interaction Time'], value_vars=['J1', 'J2', 'Jc'], var_name='Current', value_name='Heat')
fig_heat = px.line(df_melt, x='Time', y='Heat',
                   color='Interaction Time', line_dash='Current',
                   title='Heat Currents')
fig_heat.update_xaxes(range=[0, 200])
fig_heat.update_yaxes(range=[-0.005, 0.0001])
fig_heat.show()

## Symplectic Eigenvalues

In [ ]:
fig, ax = plt.subplots()
x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d1']
y2=df.loc[df['Interaction Time'] == 1.0]['d1-']
ax.plot(x, y1, y2)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d2']
y2=df.loc[df['Interaction Time'] == 1.0]['d2-']

ax.plot(x, y1, y2)
plt.show()

## Save Data Frame

In [ ]:
pd.to_pickle(df, f'../objects/saved/observables.pkl')